# Kaggle Taxi travel prediction challenge
https://www.kaggle.com/c/cs6140fall/data

# Data preprocessing

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
# get 3.x version division and print method

In [2]:
os.chdir("/Users/songzhewei/Desktop/Data Science/master course/machine learning/Mini Project/all")

In [3]:
df_train_input = pd.read_csv('train.csv',header = 0)

In [1]:
def travel_time(ployline):
    temp = [0 for _ in range(len(ployline))]
    for i in range(0,len(ployline)):
        temp[i]=ployline[i].count(',')*15
    return temp
        
    

In [5]:
df_train_input['trip_time']=travel_time(df_train_input['POLYLINE'])

In [6]:
df_test_input = pd.read_csv('test_public.csv',header = 0)

In [7]:
from datetime import datetime
A=df_train_input['TIMESTAMP']
temp2 = [0 for _ in range(len(A))]
for i in range(0,len(A)):
    temp2[i]=datetime.fromtimestamp(A[i]).strftime("%Y-%m-%d %H:%M:%S")
   

In [8]:
df_train_input['datetime']=temp2

In [9]:
def Get_Start_End(location_feature):
    temp_start = [0 for _ in range(len(location_feature))]
    temp_end = [0 for _ in range(len(location_feature))]
    for i in range(0,len(location_feature)):
        j=i
        str=location_feature[i]
        strlist = str.split(',')
        temp_start[i]=strlist[0]
        temp_end[j]=strlist[-1]
    return temp_start,temp_end

In [10]:
df_train_input['start_gps'],df_train_input['end_gps']=Get_Start_End(df_train_input['POLYLINE'])

In [19]:
# delete missing value
df_train=df_train_input[df_train_input.MISSING_DATA==False]
df_train=df_train_input.drop(columns=['MISSING_DATA'])


In [20]:
B=df_test_input['TIMESTAMP']
temp3 = [0 for _ in range(len(B))]
for j in range(0,len(B)):
    temp3[j]=datetime.fromtimestamp(B[j]).strftime("%Y-%m-%d %H:%M:%S")
df_test_input['datetime']=temp3
df_test=df_test_input[df_train_input.MISSING_DATA==False]
df_test=df_test_input.drop(columns=['MISSING_DATA'])

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [21]:
df_train['month'] = pd.DatetimeIndex(df_train.datetime).month
df_train['day'] = pd.DatetimeIndex(df_train.datetime).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train.datetime).hour
df_train['minute'] = pd.DatetimeIndex(df_train.datetime).minute

In [22]:
df_test['month'] = pd.DatetimeIndex(df_test.datetime).month
df_test['day'] = pd.DatetimeIndex(df_test.datetime).dayofweek
df_test['hour'] = pd.DatetimeIndex(df_test.datetime).hour
df_test['minute'] = pd.DatetimeIndex(df_test.datetime).minute

In [15]:
df_train.dtypes

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
POLYLINE         object
trip_time         int64
datetime         object
start_gps        object
end_gps          object
month             int64
day               int64
hour              int64
minute            int64
dtype: object

In [34]:
df_train.shape

(1710670, 16)

In [35]:
df_train.count()

TRIP_ID         1710670
CALL_TYPE       1710670
ORIGIN_CALL      364770
ORIGIN_STAND     806579
TAXI_ID         1710670
TIMESTAMP       1710670
DAY_TYPE        1710670
POLYLINE        1710670
trip_time       1710670
datetime        1710670
start_gps       1710670
end_gps         1710670
month           1710670
day             1710670
hour            1710670
minute          1710670
dtype: int64

In [23]:
base_df_stats = new_df = df_train[['trip_time','month','day','hour','minute']].groupby(by=['month', 'day','hour','minute']).agg(['mean', 'std'])
base_df_stats.columns = base_df_stats.columns.droplevel(0)
base_df_stats = base_df_stats.reset_index()
base_df_stats2 = new_df = df_train[['trip_time','month','day','hour']].groupby(by=['month', 'day','hour']).agg(['mean', 'std'])
base_df_stats2.columns = base_df_stats2.columns.droplevel(0)
base_df_stats2 = base_df_stats2.reset_index()
base_stats_join = pd.merge(base_df_stats, base_df_stats2, how='inner', on=['month', 'day','hour'])
data_df = pd.merge(df_train, base_stats_join, how='left', on=['day', 'month', 'hour', 'minute'])
# data_df=data_df.fillna(0)

In [25]:
qujian_month=[0,3,6,9,12]
qujian_hour=[0,6,11,17,24]
data_df ['month']=pd.cut(data_df .month,qujian_month,labels=['spring','summer','fall','winter'])
data_df ['hour']=pd.cut(data_df .hour,qujian_hour,labels=['midnight','morning','afternoon','night'])
qujian_minute=[0,20,40,60]
data_df['minute']=pd.cut(data_df.minute,qujian_minute,labels=['early_20','medium_20','last_20'])


In [26]:
data_df['hour']=data_df['hour'].fillna('midnight')
data_df['minute']=data_df['minute'].fillna('early_20')

In [27]:
data_df_test = pd.merge(df_test, base_stats_join, how='left', on=['day', 'month', 'hour', 'minute'])
data_df_test['month']=pd.cut(data_df_test.month,qujian_month,labels=['spring','summer','fall','winter'])
data_df_test['hour']=pd.cut(data_df_test.hour,qujian_hour,labels=['midnight','morning','afternoon','night'])
data_df_test['minute']=pd.cut(data_df_test.minute,qujian_minute,labels=['early_20','medium_20','last_20'])
data_df_test['hour']=data_df_test['hour'].fillna('midnight')
data_df_test['minute']=data_df_test['minute'].fillna('early_20')

In [28]:
data_df_test

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,datetime,month,day,hour,minute,mean_x,std_x,mean_y,std_y
0,T1,B,NaN,15.0,20000542,1408039037,A,2014-08-14 13:57:17,fall,3,afternoon,last_20,1429.000000,596.547208,1546.452569,2877.214610
1,T2,B,NaN,57.0,20000108,1408038611,A,2014-08-14 13:50:11,fall,3,afternoon,last_20,1281.923077,540.576615,1546.452569,2877.214610
2,T3,B,NaN,15.0,20000370,1408038568,A,2014-08-14 13:49:28,fall,3,afternoon,last_20,1523.181818,911.741590,1546.452569,2877.214610
3,T4,B,NaN,53.0,20000492,1408039090,A,2014-08-14 13:58:10,fall,3,afternoon,last_20,1339.615385,643.075304,1546.452569,2877.214610
4,T5,B,NaN,18.0,20000621,1408039177,A,2014-08-14 13:59:37,fall,3,afternoon,last_20,1407.631579,644.151829,1546.452569,2877.214610
5,T6,A,42612.0,NaN,20000607,1408037146,A,2014-08-14 13:25:46,fall,3,afternoon,medium_20,1691.666667,888.031531,1546.452569,2877.214610
6,T7,B,NaN,15.0,20000310,1408038846,A,2014-08-14 13:54:06,fall,3,afternoon,last_20,1555.263158,854.108492,1546.452569,2877.214610
7,T8,A,31780.0,NaN,20000619,1408038948,A,2014-08-14 13:55:48,fall,3,afternoon,last_20,1227.500000,481.949167,1546.452569,2877.214610
8,T9,B,NaN,9.0,20000503,1408038563,A,2014-08-14 13:49:23,fall,3,afternoon,last_20,1523.181818,911.741590,1546.452569,2877.214610
9,T10,B,NaN,15.0,20000327,1408038021,A,2014-08-14 13:40:21,fall,3,afternoon,medium_20,990.000000,644.551339,1546.452569,2877.214610


In [29]:
version1_test=data_df_test.ix[:,[8,9,10,11,12,13,14,15]]

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [30]:
version1=data_df.ix[:,[8,12,13,14,15,16,17,18,19]]

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [32]:
var_to_encode=['month','day','hour','minute']
one_hot_version1 = pd.get_dummies(version1, columns=var_to_encode)

In [33]:
version1_test_oneHot= pd.get_dummies(version1_test, columns=var_to_encode)

In [34]:
one_hot_version1.columns

Index(['trip_time', 'mean_x', 'std_x', 'mean_y', 'std_y', 'month_spring',
       'month_summer', 'month_fall', 'month_winter', 'day_0', 'day_1', 'day_2',
       'day_3', 'day_4', 'day_5', 'day_6', 'hour_midnight', 'hour_morning',
       'hour_afternoon', 'hour_night', 'minute_early_20', 'minute_medium_20',
       'minute_last_20'],
      dtype='object')

In [67]:
version1_test_oneHot.columns

Index(['mean_x', 'std_x', 'mean_y', 'std_y', 'month_spring', 'month_summer',
       'month_fall', 'month_winter', 'day_0', 'day_1', 'day_3', 'day_4',
       'day_6', 'hour_midnight', 'hour_morning', 'hour_afternoon',
       'hour_night', 'minute_early_20', 'minute_medium_20', 'minute_last_20'],
      dtype='object')

In [69]:
version1_test_oneHot['day_2'] = [0 for _ in range(320)]
version1_test_oneHot['day_5'] = [0 for _ in range(320)]

In [110]:
one_hot_version1=one_hot_version1.drop(['mean_x','std_x','mean_y','std_y'],axis=1)

In [111]:
temporary=one_hot_version1

In [112]:
one_hot_version1=temporary.fillna(0)

In [113]:
one_hot_version1

,trip_time,month_spring,month_summer,month_fall,month_winter,day_0,day_1,day_2,day_3,day_4,day_5,day_6,hour_midnight,hour_morning,hour_afternoon,hour_night,minute_early_20,minute_medium_20,minute_last_20
0,675,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
1,555,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
2,1935,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
3,1275,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
4,855,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
5,765,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
6,1065,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
7,1005,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
8,1125,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
9,555,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0


In [156]:
df_train_target = one_hot_version1['trip_time'].values
df_train_data = one_hot_version1.drop(['trip_time'],axis = 1).values
print ('df_train_data shape is ', df_train_data.shape)
print ('df_train_target shape is ', df_train_target.shape)

df_train_data shape is  (531549, 18)
df_train_target shape is  (531549,)


In [115]:
version1_test_oneHot=version1_test_oneHot.drop(['mean_x','std_x','mean_y','std_y'],axis=1)

In [155]:
one_hot_version1=one_hot_version1[(one_hot_version1['trip_time']<=1000)&(one_hot_version1['trip_time']>=200)]

In [116]:
df_test_data = version1_test_oneHot.values

In [134]:
X_lgb = df_train_data
y_lgb = df_train_target.reshape(df_train_target.shape[0],)

In [157]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor()
gbdt_model = clf.fit(df_train_data, df_train_target)  # Training model
y_upper = gbdt_model.predict(df_test_data)  # predict


In [158]:
commit_df = pd.DataFrame(y_upper).astype('int')
df_test['TRAVEL_TIME']=commit_df

In [159]:
submit4=df_test.ix[:,[0,12]]

In [145]:
submit4['TRAVEL_TIME']=submit4['TRAVEL_TIME']-600

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_lgb, y_lgb)
# specify your configurations as a dict
params = {
    'num_leaves': 128,
    'learning_rate':0.01,
    'n_estimators':800,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'application':'regression_l1'
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000)

Start training...


/Users/songzhewei/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [136]:
# commit_df = pd.date_range('2016/9/1', periods=30, freq='D')
# commit_df = pd.DataFrame(commit_df)
# commit_df.columns = ['predict_date']
y_predict = gbm.predict(df_test_data )
commit_df = pd.DataFrame(y_predict).astype('int')
df_test['TRAVEL_TIME']=commit_df

In [137]:
submit4=df_test.ix[:,[0,12]]

In [161]:
submit4.to_csv('submit13.csv')

In [47]:
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be r

In [61]:
# 总得切分一下数据咯（训练集和测试集）
cv = cross_validation.ShuffleSplit(len(df_train_data), n_iter=3, test_size=0.2,
    random_state=0)
print ("随机森林回归/Random Forest(n_estimators = 100)")   
for train, test in cv:    
    svc = RandomForestRegressor(n_estimators = 100).fit(df_train_data[train], df_train_target[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        svc.score(df_train_data[train], df_train_target[train]), svc.score(df_train_data[test], df_train_target[test])))

随机森林回归/Random Forest(n_estimators = 100)
train score: 0.103, test score: -0.066

train score: 0.103, test score: -0.072

train score: 0.103, test score: -0.065



In [72]:
y_predict2 = svc.predict(df_test_data)
commit_df2 = pd.DataFrame(y_predict2).astype('int')
df_test['TRAVEL_TIME']=commit_df2

In [75]:
submit5=df_test.ix[:,[0,12]]

In [76]:
submit5.to_csv('submit5.csv')

In [16]:
type_A=df_train.dropna(axis = 0, subset = ['ORIGIN_CALL'] )  #训练集中364770条有用户记录
type_B=df_train.dropna(axis=0,subset=['ORIGIN_STAND'])
df_train_TypeC=df_train.fillna('NaN')
type_C=df_train_TypeC[(df_train_TypeC['ORIGIN_CALL']=='NaN')&(df_train_TypeC['ORIGIN_STAND']=='NaN')]
type_A_test=df_test.dropna(axis = 0, subset = ['ORIGIN_CALL'] )
type_B_test=df_test.dropna(axis = 0, subset = ['ORIGIN_STAND'] )

In [17]:
df_test_TypeC=df_test.fillna('NaN')
type_C_test=df_test_TypeC[(df_test_TypeC['ORIGIN_CALL']=='NaN')&(df_test_TypeC['ORIGIN_STAND']=='NaN')]

In [84]:
type_A=type_A[(type_A['trip_time']>=500)&(type_A['trip_time']<=3500)]

In [85]:
type_A

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE,trip_time,datetime,start_gps,end_gps,month,day,hour,minute
12,1372637343620000571,A,31508.0,NaN,20000571,1372637343,A,"[[-8.618868,41.155101],[-8.6175,41.154912],[-8...",945,2013-06-30 20:09:03,[[-8.618868,41.162265]],6,6,20,9
17,1372639135620000570,A,33180.0,NaN,20000570,1372639135,A,"[[-8.666757,41.174055],[-8.666784,41.174064],[...",555,2013-06-30 20:38:55,[[-8.666757,41.181714]],6,6,20,38
21,1372637254620000657,A,39233.0,NaN,20000657,1372637254,A,"[[-8.660646,41.168574],[-8.661087,41.167926],[...",1275,2013-06-30 20:07:34,[[-8.660646,41.181813]],6,6,20,7
24,1372637658620000596,A,22864.0,NaN,20000596,1372637658,A,"[[-8.665686,41.170626],[-8.665677,41.170653],[...",765,2013-06-30 20:14:18,[[-8.665686,41.187816]],6,6,20,14
26,1372639535620000161,A,25862.0,NaN,20000161,1372639535,A,"[[-8.648226,41.148333],[-8.648514,41.148297],[...",1695,2013-06-30 20:45:35,[[-8.648226,41.097357]],6,6,20,45
36,1372640130620000010,A,4082.0,NaN,20000010,1372640130,A,"[[-8.621523,41.177673],[-8.621532,41.177682],[...",765,2013-06-30 20:55:30,[[-8.621523,41.164209]],6,6,20,55
41,1372638793620000571,A,2002.0,NaN,20000571,1372638793,A,"[[-8.574705,41.142717],[-8.574696,41.142726],[...",1215,2013-06-30 20:33:13,[[-8.574705,41.142636]],6,6,20,33
57,1372638451620000621,A,59594.0,NaN,20000621,1372638451,A,"[[-8.602209,41.163372],[-8.601651,41.163759],[...",1035,2013-06-30 20:27:31,[[-8.602209,41.193315]],6,6,20,27
70,1372641048620000560,A,40433.0,NaN,20000560,1372641048,A,"[[-8.578683,41.145777],[-8.579448,41.146488],[...",735,2013-06-30 21:10:48,[[-8.578683,41.154507]],6,6,21,10
72,1372641858620000154,A,21875.0,NaN,20000154,1372641858,A,"[[-8.623152,41.164191],[-8.623422,41.164227],[...",615,2013-06-30 21:24:18,[[-8.623152,41.160087]],6,6,21,24


In [120]:
type_C[(type_C['month']==8)&(type_C['day']==3)&(type_C['hour']==13)&(type_C['minute']==58)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE,trip_time,datetime,start_gps,end_gps,month,day,hour,minute
150117,1375379901620000429,C,NaN,NaN,20000429,1375379901,A,"[[-8.560935,41.168601]]",15,2013-08-01 13:58:21,[[-8.560935,41.168601]],8,3,13,58
207450,1376589529620000006,C,NaN,NaN,20000006,1376589529,A,"[[-8.61003,41.140962],[-8.610165,41.141016],[-...",1215,2013-08-15 13:58:49,[[-8.61003,41.15277]],8,3,13,58
208915,1376589513620000229,C,NaN,NaN,20000229,1376589513,A,"[[-8.613891,41.177673],[-8.613909,41.177673],[...",615,2013-08-15 13:58:33,[[-8.613891,41.173956]],8,3,13,58


In [19]:
# type_A_test[(type_A_test['month']==9)&(type_A_test['day']==1)&
#                     (type_A_test['hour']==4)]

In [68]:
# test_customer_match=pd.merge(type_A_test,type_A,on='ORIGIN_CALL',how='left')
# test_customer_match2=test_customer_match[['TRIP_ID_x','TAXI_ID_x','ORIGIN_CALL','datetime_x','month_x','day_x','hour_x','minute_x','TRIP_ID_y','TAXI_ID_y','datetime_y','month_y','day_y','hour_y','minute_y','trip_time']]

In [20]:
#测试集中用户在训练集中出现过的记录
# test_customer_match3=test_customer_match2.dropna(axis = 0, subset = ['TRIP_ID_y'] )
# test_customer_match3['TRIP_ID_x'].unique()
#test_customer_match customer appeared in train dataset also appear in test dataset
#test_customer_match2 提取重要特征（based on domain knowledge)
#test_customer_match3 删除训练集中与测试集不重合的记录

In [104]:
# test_stand_match=pd.merge(type_B_test,type_B,on='ORIGIN_STAND',how='left')
# test_stand_match2=test_stand_match[['TRIP_ID_x','TAXI_ID_x','ORIGIN_STAND','datetime_x','month_x','day_x','hour_x','minute_x','TRIP_ID_y','TAXI_ID_y','datetime_y','month_y','day_y','hour_y','minute_y','trip_time']]
# test_stand_match2['ORIGIN_STAND'].unique()

In [105]:
# type_B_test['ORIGIN_STAND'].unique()

In [106]:
# test_stand_match2[(test_stand_match2['ORIGIN_STAND']==15.0)&(test_stand_match2['month_y']==9)&
#                   (test_stand_match2['day_y']==1)&(test_stand_match2['hour_y']==4)&(test_stand_match2['minute_y']==19)]

In [21]:
# df_train_target1 = test_customer_match3['trip_time'].values
# df_train_customer = test_customer_match3.drop(['trip_time','TRIP_ID_x','month_x','hour_x','day_x','TRIP_ID_y',],axis = 1).values
# print('df_train_customer shape is ', df_train_customer.shape)
# print('df_train_target1 shape is ', df_train_target1.shape)

# Machine Learning Algorithms

In [37]:
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/songzhewei/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  Dep

In [87]:
base_df=type_A[['datetime','ORIGIN_CALL','month','day','hour','minute','trip_time','TAXI_ID']]

In [88]:
base_df_stats = new_df = base_df[['trip_time','month','day','hour','minute','ORIGIN_CALL','TAXI_ID']].groupby(by=['month', 'day','hour','minute','ORIGIN_CALL','TAXI_ID']).agg(['mean', 'std'])
base_df_stats.columns = base_df_stats.columns.droplevel(0)
base_df_stats = base_df_stats.reset_index()


In [89]:
base_df_stats[base_df_stats['TAXI_ID']==20000508]

,month,day,hour,minute,ORIGIN_CALL,TAXI_ID,mean,std
219,1,0,2,24,2002.0,20000508,525,NaN
502,1,0,3,33,54512.0,20000508,765,NaN
1030,1,0,4,54,29682.0,20000508,645,NaN
1422,1,0,6,3,7825.0,20000508,1725,NaN
1581,1,0,6,40,27588.0,20000508,1725,NaN
3276,1,0,14,59,9105.0,20000508,825,NaN
3958,1,1,2,16,40651.0,20000508,1695,NaN
4059,1,1,2,53,30608.0,20000508,1725,NaN
4145,1,1,3,15,13794.0,20000508,2055,NaN
4419,1,1,4,2,43288.0,20000508,1305,NaN


In [19]:
# base_df_stats2 = new_df = base_df[['trip_time','month','day','hour']].groupby(by=['month', 'day','hour']).agg(['mean', 'std'])
# base_df_stats2.columns = base_df_stats2.columns.droplevel(0)
# base_df_stats2 = base_df_stats2.reset_index()
# base_stats_join = pd.merge(base_df_stats, base_df_stats2, how='inner', on=['month', 'day','hour'])
# data_df = pd.merge(base_df, base_stats_join, how='inner', on=['day', 'month', 'hour', 'minute'])
# data_df=data_df.fillna(0)

In [172]:
# stats_df = pd.merge(type_A_test, data_df, how='left', on=['day', 'month', 'hour','ORIGIN_CALL'])
stats_df = pd.merge(type_A_test, base_df_stats, how='left', on=['month','day','hour','minute','ORIGIN_CALL'])
typeA_customer_match=stats_df.dropna(axis=0,subset=['mean'])

In [173]:
typeA_customer_match['TRIP_ID'].unique()

array(['T37', 'T64', 'T117', 'T194', 'T235', 'T260'], dtype=object)

In [179]:
typeA_customer_match[(typeA_customer_match['TRIP_ID']=='T260')]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID_x,TIMESTAMP,DAY_TYPE,datetime,month,day,hour,minute,TAXI_ID_y,mean,std
58,T260,A,2002.0,NaN,20000190,1414813685,A,2014-10-31 23:48:05,10,4,23,48,20000009.0,825.0,NaN


In [51]:
B_base_df=type_B[['datetime','ORIGIN_STAND','month','day','hour','minute','trip_time']]
B_base_df_stats = B_new_df = B_base_df[['trip_time','month','day','hour','minute','ORIGIN_STAND']].groupby(by=['month', 'day','hour','minute','ORIGIN_STAND']).agg(['mean', 'std'])
B_base_df_stats.columns = B_base_df_stats.columns.droplevel(0)
B_base_df_stats = B_base_df_stats.reset_index()

In [52]:
B_base_df_stats2 = B_new_df = B_base_df[['trip_time','month','day','hour','ORIGIN_STAND']].groupby(by=['month', 'day','hour','ORIGIN_STAND']).agg(['mean', 'std'])
B_base_df_stats2.columns = B_base_df_stats2.columns.droplevel(0)
B_base_df_stats2 = B_base_df_stats2.reset_index()
B_base_stats_join = pd.merge(B_base_df_stats, B_base_df_stats2, how='inner', on=['month', 'day','hour','ORIGIN_STAND'])
B_data_df = pd.merge(B_base_df, B_base_stats_join, how='inner', on=['day', 'month', 'hour', 'minute','ORIGIN_STAND'])

In [54]:
B_stats_df = pd.merge(type_B_test, B_data_df, how='left', on=['day', 'month', 'hour','ORIGIN_STAND'])

In [121]:
C_base_df=type_C[['datetime','month','day','hour','minute','trip_time']]
C_base_df_stats = C_new_df = C_base_df[['trip_time','month','day','hour','minute']].groupby(by=['month', 'day','hour','minute']).agg(['mean', 'std'])
C_base_df_stats.columns = C_base_df_stats.columns.droplevel(0)
C_base_df_stats = C_base_df_stats.reset_index()

In [122]:
C_base_df_stats2 = C_new_df = C_base_df[['trip_time','month','day','hour']].groupby(by=['month', 'day','hour']).agg(['mean', 'std'])
C_base_df_stats2.columns = C_base_df_stats2.columns.droplevel(0)
C_base_df_stats2 = C_base_df_stats2.reset_index()
C_base_stats_join = pd.merge(C_base_df_stats, C_base_df_stats2, how='inner', on=['month', 'day','hour'])
C_data_df = pd.merge(C_base_df, C_base_stats_join, how='inner', on=['day', 'month', 'hour', 'minute'])

In [123]:
C_stats_df = pd.merge(type_C_test, C_data_df, how='left', on=['day', 'month', 'hour'])

In [131]:
C_stats_df_fill=C_stats_df.fillna('NaN')
C_stats_df_fill[C_stats_df_fill['minute_y']==10]


,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,datetime_x,month,day,hour,minute_x,datetime_y,minute_y,trip_time,mean_x,std_x,mean_y,std_y
31,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-01 13:10:17,10,15,1891.666667,1647.48,1557.024793,1420.082851
32,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-01 13:10:57,10,855,1891.666667,1647.48,1557.024793,1420.082851
33,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-01 13:10:45,10,1125,1891.666667,1647.48,1557.024793,1420.082851
34,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-08 13:10:39,10,1065,1891.666667,1647.48,1557.024793,1420.082851
35,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-08 13:10:32,10,2385,1891.666667,1647.48,1557.024793,1420.082851
36,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-08 13:10:51,10,2325,1891.666667,1647.48,1557.024793,1420.082851
37,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-08 13:10:09,10,1575,1891.666667,1647.48,1557.024793,1420.082851
38,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-15 13:10:46,10,1875,1891.666667,1647.48,1557.024793,1420.082851
39,T12,C,NaN,NaN,20000160,1408038946,A,2014-08-14 13:55:46,8,3,13,55,2013-08-29 13:10:09,10,5805,1891.666667,1647.48,1557.024793,1420.082851
273,T13,C,NaN,NaN,20000017,1408039130,A,2014-08-14 13:58:50,8,3,13,58,2013-08-01 13:10:17,10,15,1891.666667,1647.48,1557.024793,1420.082851


In [132]:
TRIP_ID='T400'
same_value=10
temp_predict_C = [0 for _ in range(len(C_stats_df_fill))]
for i in range(0,len(C_stats_df_fill)):
    if C_stats_df_fill.iloc[i]['TRIP_ID']==TRIP_ID:
        temp_predict_C[i]=same_value
    else:
        minute_x=C_stats_df_fill.iloc[i]['minute_x']
        temp_filter=C_stats_df_fill[(C_stats_df_fill['month']==C_stats_df_fill.iloc[i]['month'])&
                                    (C_stats_df_fill['day']==C_stats_df_fill.iloc[i]['day'])&
                                    (C_stats_df_fill['hour']==C_stats_df_fill.iloc[i]['hour'])&
                                    (C_stats_df_fill['minute_y']==minute_x)]
        while temp_filter.empty:
                minute_x=minute_x+1
                temp_filter=C_stats_df_fill[(C_stats_df_fill['month']==C_stats_df_fill.iloc[i]['month'])&
                                    (C_stats_df_fill['day']==C_stats_df_fill.iloc[i]['day'])&
                                    (C_stats_df_fill['hour']==C_stats_df_fill.iloc[i]['hour'])&
                                    (C_stats_df_fill['minute_y']==minute_x)]
                if minute_x>=60:
                    break
        minute_x=C_stats_df_fill['minute_x']
        while temp_filter.empty:        
                minute_x=minute_x-1
                temp_filter=C_stats_df_fill[(C_stats_df_fill['month']==C_stats_df_fill.iloc[i]['month'])&
                                    (C_stats_df_fill['day']==C_stats_df_fill.iloc[i]['day'])&
                                    (C_stats_df_fill['hour']==C_stats_df_fill.iloc[i]['hour'])&
                                    (C_stats_df_fill['minute_y']==minute_x)]
        temp_predict_C[i]=temp_filter.iloc[0]['mean_x']
        same_value=temp_predict_C[i]
        print(same_value)
        TRIP_ID=C_stats_df_fill.iloc[i]['TRIP_ID']
        print(TRIP_ID)

1335.0
T12
615.0
T13
1891.6666666666667
T14
2085.0
T15
1200.0
T16
1191.0
T17
1665.0
T18
635.0
T20
1537.5
T33
1472.142857142857
T34
2947.5
T35
3573.0
T36
615.0
T42
1537.5
T43
879.0
T47
1335.0
T48
1145.0
T59
1285.0
T62
1335.0
T63
2580.0
T69
963.0
T71
2827.5
T72
1770.0
T74
2135.0
T76
505.0
T78
2135.0
T79
5880.0
T80
1407.0
T87
505.0
T88
1407.0
T93
4065.0
T94
1485.0
T98
1710.0
T102
2385.0
T107
1761.0
T121
1710.0
T124
975.0
T131
505.0
T132
1581.0
T134
1012.5
T138
2135.0
T139
2135.0
T140
2385.0
T141
1615.0
T146
1581.0
T154
1950.0
T159
1545.0
T161
1417.5
T169
1162.5
T174
2259.0
T176
2196.4285714285716
T180
2259.0
T183
3639.0
T191
2259.0
T193
2535.0
T197
2253.0
T201
1260.0
T203
2259.0
T204
1235.0
T206
6086.25
T208
2070.0
T211
1875.0
T212
1923.0
T217
1395.0
T220
2313.0
T228
1665.0
T229
1027.5
T231
2196.4285714285716
T234
829.2857142857143
T237
1152.2727272727273
T239
1433.5714285714287
T240
1152.2727272727273
T241
829.2857142857143
T242
910.2631578947369
T243
1146.8181818181818
T244
1152.2727272

In [133]:
C_stats_df_fill['predict']=temp_predict_C
stats_TypeC_predict=C_stats_df_fill.drop_duplicates(['TRIP_ID'])
stats_TypeC_predict=stats_TypeC_predict[['TRIP_ID','predict']]
stats_TypeC_predict

,TRIP_ID,predict
0,T12,1335.000000
242,T13,615.000000
484,T14,1891.666667
726,T15,2085.000000
968,T16,1200.000000
1210,T17,1191.000000
1452,T18,1665.000000
1694,T20,635.000000
1936,T33,1537.500000
2178,T34,1472.142857


In [78]:
B_stats_df_fill=B_stats_df.fillna('NaN')
B_stats_df_fill[B_stats_df_fill['minute_y']=='NaN']
#T82 T258 没有预测

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,datetime_x,month,day,hour,minute_x,datetime_y,minute_y,trip_time,mean_x,std_x,mean_y,std_y
1622,T82,B,NaN,59.0,20000617,1412065511,A,2014-09-30 04:25:11,9,1,4,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3070,T258,B,NaN,10.0,20000239,1414814326,A,2014-10-31 23:58:46,10,4,23,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
extra_T82=type_B[(type_B['month']==9)&(type_B['day']==1)&(type_B['hour']==4)&(type_B['minute']==25)]

In [140]:
np.mean(extra_T82['trip_time'])

1534.2857142857142

In [142]:
extra_T258=type_B[(type_B['month']==10)&(type_B['day']==4)&(type_B['hour']==23)&(type_B['minute']==58)]
np.mean(extra_T258['trip_time'])

1149.0

In [79]:
B_stats_df_notnull=B_stats_df_fill[B_stats_df_fill['minute_y']!='NaN']

In [91]:
B_stats_df_notnull['TRIP_ID'].unique()

array(['T1', 'T2', 'T3', 'T4', 'T5', 'T7', 'T9', 'T10', 'T11', 'T19',
       'T21', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T30', 'T31',
       'T32', 'T38', 'T39', 'T40', 'T41', 'T44', 'T45', 'T46', 'T49',
       'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57', 'T58',
       'T61', 'T65', 'T66', 'T67', 'T70', 'T73', 'T75', 'T77', 'T85',
       'T90', 'T96', 'T103', 'T104', 'T109', 'T110', 'T112', 'T113',
       'T114', 'T118', 'T122', 'T123', 'T125', 'T130', 'T135', 'T136',
       'T137', 'T142', 'T143', 'T147', 'T149', 'T153', 'T155', 'T156',
       'T162', 'T163', 'T166', 'T167', 'T171', 'T173', 'T177', 'T178',
       'T179', 'T181', 'T182', 'T184', 'T188', 'T189', 'T192', 'T195',
       'T196', 'T199', 'T202', 'T205', 'T207', 'T210', 'T213', 'T214',
       'T218', 'T219', 'T221', 'T222', 'T224', 'T225', 'T226', 'T227',
       'T232', 'T233', 'T236', 'T238', 'T247', 'T250', 'T254', 'T257',
       'T262', 'T273', 'T281', 'T283', 'T284', 'T291', 'T298', 'T303',
       'T305'

In [101]:
B_stats_df_notnull[B_stats_df_notnull['TRIP_ID']=='T96']

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,datetime_x,month,day,hour,minute_x,datetime_y,minute_y,trip_time,mean_x,std_x,mean_y,std_y
1654,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-03 03:02:59,2,705,705,NaN,1531.67,1311.37
1655,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-03 03:55:16,55,4725,2910,2566.8,1531.67,1311.37
1656,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-24 03:55:26,55,1095,2910,2566.8,1531.67,1311.37
1657,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-17 03:19:17,19,1485,1485,NaN,1531.67,1311.37
1658,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-17 03:42:43,42,2085,2085,NaN,1531.67,1311.37
1659,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-24 03:32:34,32,585,585,NaN,1531.67,1311.37
1660,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-24 03:09:43,9,915,915,NaN,1531.67,1311.37
1661,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-24 03:58:30,58,1695,1695,NaN,1531.67,1311.37
1662,T96,B,NaN,60.0,20000688,1412062316,A,2014-09-30 03:31:56,9,1,3,31,2013-09-24 03:36:56,36,495,495,NaN,1531.67,1311.37


In [99]:
TRIP_ID='T400'
same_value=10
temp_predict_B = [0 for _ in range(len(B_stats_df_notnull))]
for i in range(0,len(B_stats_df_notnull)):
    if B_stats_df_notnull.iloc[i]['TRIP_ID']==TRIP_ID:
        temp_predict_B[i]=same_value
    else:
        minute_x=B_stats_df_notnull.iloc[i]['minute_x']
        temp_filter=B_stats_df_notnull[(B_stats_df_notnull['ORIGIN_STAND']==B_stats_df_notnull.iloc[i]['ORIGIN_STAND'])&
                                       (B_stats_df_notnull['month']==B_stats_df_notnull.iloc[i]['month'])&(B_stats_df_notnull['day']==B_stats_df_notnull.iloc[i]['day'])&
                                       (B_stats_df_notnull['hour']==B_stats_df_notnull.iloc[i]['hour'])&(B_stats_df_notnull['minute_y']==minute_x)]
        while temp_filter.empty:
                minute_x=minute_x+1
                temp_filter=B_stats_df_notnull[(B_stats_df_notnull['ORIGIN_STAND']==B_stats_df_notnull.iloc[i]['ORIGIN_STAND'])&
                                           (B_stats_df_notnull['month']==B_stats_df_notnull.iloc[i]['month'])&(B_stats_df_notnull['day']==B_stats_df_notnull.iloc[i]['day'])&
                                           (B_stats_df_notnull['hour']==B_stats_df_notnull.iloc[i]['hour'])&(B_stats_df_notnull['minute_y']==minute_x)]
                if minute_x>=60:
                    break
        minute_x=B_stats_df_notnull.iloc[i]['minute_x']
        while temp_filter.empty:        
                minute_x=minute_x-1
                temp_filter=B_stats_df_notnull[(B_stats_df_notnull['ORIGIN_STAND']==B_stats_df_notnull.iloc[i]['ORIGIN_STAND'])&
                                           (B_stats_df_notnull['month']==B_stats_df_notnull.iloc[i]['month'])&(B_stats_df_notnull['day']==B_stats_df_notnull.iloc[i]['day'])&
                                           (B_stats_df_notnull['hour']==B_stats_df_notnull.iloc[i]['hour'])&(B_stats_df_notnull['minute_y']==minute_x)]
        temp_predict_B[i]=temp_filter.iloc[0]['mean_x']
        same_value=temp_predict_B[i]
        print(same_value)
        TRIP_ID=B_stats_df_notnull.iloc[i]['TRIP_ID']
        print(TRIP_ID)

1795.0
T1
1125.0
T2
1656.4285714285713
T3
1005.0
T4
1665.0
T5
1215.0
T7
1245.0
T9
1935.0
T10
495.0
T11
1770.0
T19
1851.0
T21
1165.0
T24
1425.0
T25
1275.0
T26
1215.0
T27
1165.0
T28
675.0
T29
735.0
T30
1620.0
T31
2085.0
T32
675.0
T38
1851.0
T39
1365.0
T40
2115.0
T41
1875.0
T44
1215.0
T45
705.0
T46
1545.0
T49
495.0
T50
2985.0
T51
1365.0
T52
675.0
T53
1656.4285714285713
T54
735.0
T55
975.0
T56
1590.0
T57
1410.0
T58
1965.0
T61
2445.0
T65
1875.0
T66
1395.0
T67
1035.0
T70
1795.0
T73
765.0
T75
1995.0
T77
1185.0
T85
1065.0
T90
585.0
T96
2745.0
T103
15.0
T104
675.0
T109
735.0
T110
1965.0
T112
885.0
T113
855.0
T114
1140.0
T118
945.0
T122
975.0
T123
975.0
T125
1335.0
T130
1200.0
T135
2895.0
T136
465.0
T137
1515.0
T142
885.0
T143
735.0
T147
2235.0
T149
1785.0
T153
1725.0
T155
675.0
T156
1965.0
T162
2055.0
T163
2385.0
T166
1245.0
T167
1005.0
T171
1035.0
T173
2445.0
T177
1635.0
T178
2655.0
T179
1905.0
T181
1470.0
T182
3285.0
T184
1815.0
T188
885.0
T189
285.0
T192
975.0
T195
885.0
T196
2355.0
T199
544

In [102]:
B_stats_df_notnull['predict']=temp_predict_B
stats_TypeB_predict=B_stats_df_notnull.drop_duplicates(['TRIP_ID'])
stats_TypeB_predict=stats_TypeB_predict[['TRIP_ID','predict']]
stats_TypeB_predict

/Users/songzhewei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TRIP_ID,predict
0,T1,1795.000000
64,T2,1125.000000
105,T3,1656.428571
169,T4,1005.000000
199,T5,1665.000000
223,T7,1215.000000
287,T9,1245.000000
320,T10,1935.000000
384,T11,495.000000
394,T19,1770.000000


In [21]:
type_A_test['TRIP_ID'].unique()

array(['T6', 'T8', 'T22', 'T23', 'T37', 'T60', 'T64', 'T68', 'T81', 'T83',
       'T84', 'T86', 'T91', 'T92', 'T95', 'T99', 'T100', 'T101', 'T111',
       'T115', 'T116', 'T117', 'T119', 'T120', 'T126', 'T127', 'T128',
       'T129', 'T133', 'T144', 'T145', 'T148', 'T151', 'T152', 'T157',
       'T158', 'T160', 'T164', 'T168', 'T170', 'T172', 'T175', 'T185',
       'T186', 'T187', 'T190', 'T194', 'T198', 'T200', 'T209', 'T215',
       'T216', 'T223', 'T230', 'T235', 'T251', 'T260', 'T293', 'T300',
       'T302', 'T304', 'T310', 'T312', 'T313', 'T314', 'T315', 'T317',
       'T319', 'T320', 'T323', 'T326', 'T327'], dtype=object)

In [32]:
TRIP_ID='T400'
same_value=10
temp_predict = [0 for _ in range(len(stats_df))]
for i in range(0,len(stats_df)):
    if stats_df.iloc[i]['TRIP_ID']==TRIP_ID:
        temp_predict[i]=same_value
    else:
        minute_x=stats_df.iloc[i]['minute_x']
        temp_filter=stats_df[(stats_df['month']==stats_df.iloc[i]['month'])&(stats_df['day']==stats_df.iloc[i]['day'])&
                    (stats_df['hour']==stats_df.iloc[i]['hour'])&(stats_df['minute_y']==minute_x)]
        while temp_filter.empty:
            minute_x=minute_x+1
            temp_filter=stats_df[(stats_df['month']==stats_df.iloc[i]['month'])&(stats_df['day']==stats_df.iloc[i]['day'])&
                        (stats_df['hour']==stats_df.iloc[i]['hour'])&(stats_df['minute_y']==minute_x)]
        temp_predict[i]=temp_filter.iloc[0]['mean_x']
        same_value=temp_predict[i]
        print(same_value)
        TRIP_ID=stats_df.iloc[i]['TRIP_ID']
        print(TRIP_ID)
        

1705.0
T6
1825.0
T8
1825.0
T22
1825.0
T23
2505.0
T37
915.0
T60
1683.0
T64
645.0
T68
1431.0
T81
1975.0
T83
1215.0
T84
1146.4285714285713
T86
1975.0
T91
1905.0
T92
1101.0
T95
1215.0
T99
1275.0
T100
1479.5454545454545
T101
1710.0
T111
1101.0
T115
1101.0
T116
1431.0
T117
1635.0
T119
1101.0
T120
1431.0
T126
1290.0
T127
1881.0
T128
1479.5454545454545
T129
1431.0
T133
1910.0
T144
1698.75
T145
1792.5
T148
1635.0
T151
1479.5454545454545
T152
1710.0
T157
1125.0
T158
1125.0
T160
1710.0
T164
1305.0
T168
1945.0
T170
1660.0
T172
1660.0
T175
1710.0
T185
2635.0
T186
1945.0
T187
3005.0
T190
1945.0
T194
1545.0
T198
2075.0
T200
1125.0
T209
1710.0
T215
1125.0
T216
1845.0
T223
2535.0
T230
1545.0
T235
1985.0
T251
635.0
T260
2065.0
T293
780.0
T300
1300.7142857142858
T302
1015.0
T304
1281.0
T310
975.0
T312
975.0
T313
2565.0
T314
955.0
T315
975.0
T317
930.0
T319
2467.5
T320
1455.0
T323
1281.0
T326
2175.0
T327


In [37]:
stats_df['predict']=temp_predict
stats_TypeA_predict=stats_df.drop_duplicates(['TRIP_ID'])
stats_TypeA_predict=stats_TypeA_predict[['TRIP_ID','predict']]
stats_TypeA_predict

,TRIP_ID,predict
0,T6,1705.000000
146,T8,1825.000000
292,T22,1825.000000
438,T23,1825.000000
584,T37,2505.000000
730,T60,915.000000
876,T64,1683.000000
1022,T68,645.000000
1168,T81,1431.000000
1513,T83,1975.000000


In [136]:
baseline_submit=stats_TypeA_predict.append(stats_TypeB_predict)
baseline_submit=baseline_submit.append(stats_TypeC_predict)
baseline_submit.to_csv('submit1.csv')